In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AutoML natural language text classification model


## Installation

Install the latest version of AutoML SDK.

In [ ]:
! pip3 install google-cloud-automl


Install the Google *cloud-storage* library as well.

In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AutoML SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AutoML APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AutoML Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID**, try to get your project ID using `gcloud` command by executing the second cell below.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AutoML. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AutoML. Not all regions provide support for all AutoML services. For the latest support per region, see [Region support for AutoML services]()

In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AutoML Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AutoML notebook and run the cell, the cell knows to skip executing the authentication steps.*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets. 

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME
    

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME
    

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

#### Import AutoML SDK

Import the AutoM SDK into our Python environment.

In [ ]:
import json
import time


from google.cloud import automl


from google.protobuf.json_format import MessageToJson


#### AutoML constants

Setup up the following constants for AutoML:

- `PARENT`: The AutoM location root path for dataset, model and endpoint resources.

In [ ]:
# AutoM location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


## Clients

The AutoML SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AutoML).

You will use several clients in this tutorial, so set them all up upfront.


In [ ]:
def automl_client():
    return automl.AutoMlClient()


def prediction_client():
    return automl.PredictionServiceClient()


def operations_client():
    return automl.AutoMlClient()._transport.operations_client


clients = {}
clients["automl"] = automl_client()
clients["prediction"] = prediction_client()
clients["operations"] = operations_client()

for client in clients.items():
    print(client)


In [ ]:
IMPORT_FILE = 'gs://cloud-ml-data/NL-classification/happiness.csv'


In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10 


*Example output*:
```
I went on a successful date with someone I felt sympathy and connection with.,affection
I was happy when my son got 90% marks in his examination,affection
I went to the gym this morning and did yoga.,exercise
We had a serious talk with some friends of ours who have been flaky lately. They understood and we had a good evening hanging out.,bonding
I went with grandchildren to butterfly display at Crohn Conservatory,affection
I meditated last night.,leisure
"I made a new recipe for peasant bread, and it came out spectacular!",achievement
I got gift from my elder brother which was really surprising me,affection
YESTERDAY MY MOMS BIRTHDAY SO I ENJOYED,enjoy_the_moment
Watching cupcake wars with my three teen children,affection
```



## Create a dataset

### [projects.locations.datasets.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request

In [ ]:
dataset = {
    "display_name": "happiness_" + TIMESTAMP,
    "text_classification_dataset_metadata": {
      "classification_type": "MULTICLASS"
    }
}

print(MessageToJson(
    automl.CreateDatasetRequest(
        parent=PARENT, 
        dataset=dataset
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "happiness_20210228224317",
    "textClassificationDatasetMetadata": {
      "classificationType": "MULTICLASS"
    }
  }
}
```



#### Call

In [ ]:
request = clients["automl"].create_dataset(
    parent=PARENT, 
    dataset=dataset
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/TCN2705019056410329088"
}
```



In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)


### [projects.locations.datasets.importData](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.datasets/importData)

#### Request

In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [IMPORT_FILE]
    }
}

print(MessageToJson(
    automl.ImportDataRequest(
        name=dataset_id,
        input_config=input_config
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/TCN2705019056410329088",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://cloud-ml-data/NL-classification/happiness.csv"
      ]
    }
  }
}
```



#### Call

In [ ]:
request = clients["automl"].import_data(
    name=dataset_id, 
    input_config=input_config
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```



## Train a model

### [projects.locations.models.create](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/create)

#### Request

In [ ]:
model = automl.Model(
    display_name="happiness_" + TIMESTAMP,
    dataset_id=dataset_short_id,
    text_classification_model_metadata=automl.TextClassificationModelMetadata()
)

print(MessageToJson(
    automl.CreateModelRequest(
        parent=PARENT, 
        model=model            
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "happiness_20210228224317",
    "datasetId": "TCN2705019056410329088",
    "textClassificationModelMetadata": {}
  }
}
```



#### Call

In [ ]:
request = clients["automl"].create_model(
    parent=PARENT,
    model=model
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TCN5333697920992542720"
}
```



In [ ]:
# The full unique ID for the training pipeline
model_id = result.name
# The short numeric ID for the training pipeline
model_short_id = model_id.split('/')[-1]

print(model_short_id)


## Evaluate the model

### [projects.locations.models.modelEvaluations.list](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/list)

#### Call

In [ ]:
request = clients["automl"].list_model_evaluations(
    parent=model_id, 
    filter=""
)


#### Response

In [ ]:
evaluations_list = [
    json.loads(MessageToJson(me.__dict__["_pb"])) 
    for me in request.model_evaluation
]

print(json.dumps(evaluations_list, indent=2))
# The  evaluation slice
evaluation_slice = request.model_evaluation[0].name


*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/TCN5333697920992542720/modelEvaluations/1436745357261371663",
    "annotationSpecId": "3130761503557287936",
    "createTime": "2021-03-01T02:56:28.878044Z",
    "evaluatedExampleCount": 1193,
    "classificationEvaluationMetrics": {
      "auPrc": 0.99065405,
      "confidenceMetricsEntry": [
        {
          "recall": 1.0,
          "precision": 0.01424979,
          "f1Score": 0.028099174
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 1.0,
          "precision": 0.5862069,
          "f1Score": 0.73913044
        },
        {
          "confidenceThreshold": 0.94,
          "recall": 0.64705884,
          "precision": 1.0,
          "f1Score": 0.7857143
        },
        
        # REMOVED FOR BREVITY
        
        {
          "confidenceThreshold": 0.999,
          "recall": 0.21372032,
          "precision": 1.0,
          "f1Score": 0.35217392
        },
        {
          "confidenceThreshold": 1.0,
          "recall": 0.0026385225,
          "precision": 1.0,
          "f1Score": 0.005263158
        }
      ],
      "logLoss": 0.14686257
    },
    "displayName": "achievement"
  }
]
```



### [projects.locations.models.modelEvaluations.get](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models.modelEvaluations/get)

#### Call

In [ ]:
request = clients["automl"].get_model_evaluation(
   name=evaluation_slice
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TCN5333697920992542720/modelEvaluations/1436745357261371663",
  "annotationSpecId": "3130761503557287936",
  "createTime": "2021-03-01T02:56:28.878044Z",
  "evaluatedExampleCount": 1193,
  "classificationEvaluationMetrics": {
    "auPrc": 0.99065405,
    "confidenceMetricsEntry": [
      {
        "recall": 1.0,
        "precision": 0.01424979,
        "f1Score": 0.028099174
      },
      {
        "confidenceThreshold": 0.05,
        "recall": 1.0,
        "precision": 0.5862069,
        "f1Score": 0.73913044
      },
      
      # REMOVED FOR BREVITY
      
      {
        "confidenceThreshold": 0.999,
        "recall": 0.23529412,
        "precision": 1.0,
        "f1Score": 0.3809524
      },
      {
        "confidenceThreshold": 1.0,
        "precision": 1.0
      }
    ],
    "logLoss": 0.005436425
  },
  "displayName": "exercise"
}
```



## Make batch predictions

### Prepare files for batch prediction

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(',')

print(test_item, test_label)


In [ ]:
import tensorflow as tf
import json


test_item_uri = "gs://" + BUCKET_NAME + "/test.txt"
with tf.io.gfile.GFile(test_item_uri, 'w') as f:
    f.write(test_item + '\n')

gcs_input_uri = "gs://" + BUCKET_NAME + "/batch.csv"
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    f.write(test_item_uri + '\n')
    

    

In [ ]:
! gsutil cat $gcs_input_uri
! gsutil cat $test_item_uri


*Example output*:
```
gs://migration-ucaip-trainingaip-20210228224317/test.txt
I went on a successful date with someone I felt sympathy and connection with.
```



### [projects.locations.models.batchPredict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/batchPredict)

#### Request

In [ ]:
input_config = {
    "gcs_source": {
        "input_uris": [gcs_input_uri]
    }
}

output_config = {
    "gcs_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
    }
}

print(MessageToJson(
    automl.BatchPredictRequest(
        name=model_id,
        input_config=input_config,
        output_config=output_config
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TCN5333697920992542720",
  "inputConfig": {
    "gcsSource": {
      "inputUris": [
        "gs://migration-ucaip-trainingaip-20210228224317/batch.csv"
      ]
    }
  },
  "outputConfig": {
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228224317/batch_output/"
    }
  }
}
```



#### Call

In [ ]:
request = clients["prediction"].batch_predict(
    name=model_id,
    input_config=input_config,
    output_config=output_config
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```

In [ ]:
destination_uri = output_config["gcs_destination"]["output_uri_prefix"][:-1]

! gsutil ls $destination_uri/*
! gsutil cat $destination_uri/prediction*/*.jsonl


*Example output*:
```
gs://migration-ucaip-trainingaip-20210228224317/batch_output/prediction-happiness_20210228224317-2021-03-01T02:57:02.004934Z/text_classification_1.jsonl
gs://migration-ucaip-trainingaip-20210228224317/batch_output/prediction-happiness_20210228224317-2021-03-01T02:57:02.004934Z/text_classification_2.jsonl
{"textSnippet":{"contentUri":"gs://migration-ucaip-trainingaip-20210228224317/test.txt"},"annotations":[{"annotationSpecId":"5436604512770981888","classification":{"score":0.93047273},"displayName":"affection"},{"annotationSpecId":"3707222255860711424","classification":{"score":0.002518793},"displayName":"achievement"},{"annotationSpecId":"7742447521984675840","classification":{"score":1.3182563E-4},"displayName":"enjoy_the_moment"},{"annotationSpecId":"824918494343593984","classification":{"score":0.06613126},"displayName":"bonding"},{"annotationSpecId":"1977839998950440960","classification":{"score":1.5267624E-5},"displayName":"leisure"},{"annotationSpecId":"8318908274288099328","classification":{"score":8.887557E-6},"displayName":"nature"},{"annotationSpecId":"3130761503557287936","classification":{"score":7.2130124E-4},"displayName":"exercise"}]}
```

## Make online predictions

### [projects.locations.models.deploy](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/deploy)

#### Call

In [ ]:
request = clients["automl"].deploy_model(name=model_id)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result))


*Example output*:
```
{}
```

### [projects.locations.models.predict](https://cloud.google.com/automl/docs/reference/rest/v1beta1/projects.locations.models/predict)

### Prepare data item for online prediction

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(',')


#### Request

In [ ]:
payload = {
    "text_snippet": {
        "content": test_item, 
        "mime_type": "text/plain"
    }
}

request = automl.PredictRequest(
    name=model_id,
    payload=payload
)

print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/TCN5333697920992542720",
  "payload": {
    "textSnippet": {
      "content": "I went on a successful date with someone I felt sympathy and connection with.",
      "mimeType": "text/plain"
    }
  }
}
```



#### Call

In [ ]:
request = clients["prediction"].predict(request=request)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "payload": [
    {
      "annotationSpecId": "5436604512770981888",
      "classification": {
        "score": 0.9272586
      },
      "displayName": "affection"
    },
    {
      "annotationSpecId": "824918494343593984",
      "classification": {
        "score": 0.068884976
      },
      "displayName": "bonding"
    },
    {
      "annotationSpecId": "3707222255860711424",
      "classification": {
        "score": 0.0028119811
      },
      "displayName": "achievement"
    },
    {
      "annotationSpecId": "3130761503557287936",
      "classification": {
        "score": 0.0008869726
      },
      "displayName": "exercise"
    },
    {
      "annotationSpecId": "7742447521984675840",
      "classification": {
        "score": 0.00013229548
      },
      "displayName": "enjoy_the_moment"
    },
    {
      "annotationSpecId": "1977839998950440960",
      "classification": {
        "score": 1.5584701e-05
      },
      "displayName": "leisure"
    },
    {
      "annotationSpecId": "8318908274288099328",
      "classification": {
        "score": 9.5975e-06
      },
      "displayName": "nature"
    }
  ]
}
```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_dataset = True
delete_model = True
delete_bucket = True

# Delete the dataset using the AutoML fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['automl'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AutoML fully qualified identifier for the model
try:
    if delete_model:
        clients['automl'].delete_model(name=model_id)
except Exception as e:
    print(e)


if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
        